<a href="https://colab.research.google.com/github/MarkNCI/Machine-Learning/blob/master/Deep%20Learning%20and%20NN/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Convolutional Neural Network
# Data that doesn't align into columns
# Images, machine translation,sentence classification, sentiment analysis
# Source data needs proper dimensions (e.g. width*height*colour channels)
# Conv2D layer convulates a 2D layer
# MaxPooling2D layes reduce data by taking a max value in a block
# Flattening will convert 2D to 1D to pass to hidden layers
# Usually Conv2D -> MaxPooling2D -> Dropout -> Flatten -> Dense -> Dropout -> Softmax

In [0]:
# CNN with MNIST
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import backend as K

In [3]:
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step
